In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
##here we will program an heston model
#stochastic volatility model
###general formula:
### dSt = (r-q)St*dt + sqrt(V)*dZs
### dV = a(Vl-V)dt + xi*V**(alpha) * dZv

def monte_carlo_heston(S0: float, a: float, alpha: float, xi: float, V0: float, VL: float, r: float, q: float, T: float, NoOfSteps: int, NoOfPaths: int, seed:int = 42) -> pd.DataFrame:
    """
    :param S0: Initial stock price
    :param a: mean revert rate
    :param alpha: exponent of Vol
    :param xi: volatility of volatility (kurtosis)
    :param V0: initial volatility
    :param VL: Long term Variance
    :param r: riskless rate
    :param rho: correlation coefficient between returns and vol
    :param q: dividend yield
    :param T: Time to maturity
    :param NoOfSteps: Number of steps on each random walk
    :param NoOfPaths: Number of random walks
    :return: Stochastic Volatility paths and Stock paths
    """
    np.random.seed(seed)
    dt = T/NoOfSteps
    drift = r - q
    cov = np.array([[1, rho],
                    [rho, 1]])
    # sampling correlated brownian motions under risk-neutral measure
    correlated_BM = np.random.multivariate_normal([0, 0], cov, (NoOfSteps, NoOfPaths))
    samples_S = correlated_BM[:, :, 0]
    paths_S = np.ones(shape=[NoOfSteps, NoOfPaths])
    samples_V = correlated_BM[:, :, 1]
    paths_V = np.ones(shape=[NoOfSteps, NoOfPaths])
    paths_V[0, :] = V0
    paths_S[0, :] = S0
    for i in range(1, NoOfSteps):
        for n in range(NoOfPaths):
            paths_V[i, n] = np.maximum(paths_V[i - 1, n] + a*(VL - paths_V[i-1, n])*dt + xi*(paths_V[i-1, n]**alpha) * samples_V[i, n]*np.sqrt(dt), 0)
            paths_S[i, n] = paths_S[i-1, n]*np.exp(drift*dt + np.sqrt(paths_V[i-1, n]) * samples_S[i, n] * np.sqrt(dt))
    vol = [f"vol_{i}" for i in range(NoOfPaths)]
    stock = [f"stock process_{i}" for i in range(NoOfPaths)]
    vol = pd.DataFrame(data=paths_V, columns = vol)
    S = pd.DataFrame(data=paths_S, columns = stock)
    return vol, S

In [18]:
def MSplot(data, p=4):
    x = data.abs()
    fig, axs = plt.subplots(nrows=int(p/2), ncols=2, figsize=(8, 6))
    r = []
    for i in range(1, p + 1):
        y = x**i
        S = y.cumsum()
        M = y.cummax()
        r.append(M/S)
    
    axs[0, 0].plot(range(len(x)), r[0], color="orangered")
    axs[0, 0].set_title("MS plot, p=1")
    axs[0, 0].set_xlabel("n")
    axs[0, 0].set_ylabel("Rn")
    axs[0, 1].plot(range(len(x)), r[1], color ="orangered")
    axs[0, 1].set_xlabel("n")
    axs[0, 1].set_title("MS plot, p=2")
    axs[1, 0].plot(range(len(x)), r[2], color="orangered")
    axs[1, 0].set_xlabel("n")
    axs[1, 0].set_ylabel("Rn")
    axs[1, 0].set_title("MS plot, p=3")
    axs[1, 1].plot(range(len(x)), r[3], color="orangered")
    axs[1, 1].set_xlabel("n")
    axs[1, 1].set_title("MS plot, p=4")
    
    plt.tight_layout()
    
        
    plt.show()
        